# Nuclio - Training function

## Environment

In [22]:
# nuclio: ignore
import nuclio

### Configurations

In [81]:
%%nuclio config

# Trigger
spec.triggers.retrain.kind = "cron"
spec.triggers.retrain.attributes.interval = "1h"

# Base image
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.retrain.kind to 'cron'
%nuclio: setting spec.triggers.retrain.attributes.interval to '1h'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


In [39]:
%nuclio mount /User ~/

mounting volume path /User as ~/


### Commands

In [91]:
%%nuclio cmd -c

# apt-get update && apt-get install -y libaio1
# apt-get install libgomp1 

############
# installs #
############

# Igz DB
pip install v3io_frames

# Utils
pip install 'fsspec>=0.3.3'
# pip install PyYAML==5.1.2
# pip install pyarrow==0.15.1
pip install pandas==0.25.3
# pip install kubernetes==9.0.0

# Function
# pip install scikit-learn==0.21.3
pip install dask-kubernetes
pip install dask-ml["complete"]==1.0.0
pip install dask-xgboost==0.1.7


### Variables

In [36]:
# DB Config
%nuclio env %v3io

In [35]:
%%nuclio env

# Features
FEATURES_TABLE=/v3io/bigdata/netops_features_parquet
# FEATURES_TABLE=netops_features
FROM_TSDB=0

# Training
TRAIN_ON_LAST=1d
TRAIN_SIZE=0.7

# Parallelizem
NUMBER_OF_SHARDS=4

# Model
MODEL_FILENAME=netops.v3.model
SAVE_TO=/v3io/bigdata/netops/models

%nuclio: setting 'FEATURES_TABLE' environment variable
%nuclio: setting '# FEATURES_TABLE' environment variable
%nuclio: setting 'FROM_TSDB' environment variable
%nuclio: setting 'TRAIN_ON_LAST' environment variable
%nuclio: setting 'TRAIN_SIZE' environment variable
%nuclio: setting 'NUMBER_OF_SHARDS' environment variable
%nuclio: setting 'MODEL_FILENAME' environment variable
%nuclio: setting 'SAVE_TO' environment variable


%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line


## Function

### Imports

In [89]:
# Utils
import os
import time
import yaml
import pandas as pd
import datetime
import itertools
import pickle

# DB Connection
import v3io_frames as v3f

# Parallelization
import dask.dataframe as dd
from dask.distributed import Client

# Function
import dask_ml.model_selection as dcv
import xgboost as xgb

### Helper functions

In [27]:
def format_df_from_tsdb(context, df):
    df.index.names = ['timestamp', 'company', 'data_center', 'device']
    df = df.reset_index()
    df = dd.from_pandas(df, npartitions=context.shards)
    return df

In [28]:
def get_data_tsdb(context):
    df = context.v3f.read(backend='tsdb', query=f'select * from {context.features_table}',
                          start=f'now-{context.train_on_last}', end='now', multi_index=True)
    df = df.reset_index(drop=True)
    df = df[sorted(df.columns)]
    df = dd.from_pandas(df, npartitions=context.shards)
    return df

In [29]:
def get_data_parquet(context):
    # Get parquet files
    mpath = [os.path.join(context.features_table, file) for file in os.listdir(context.features_table)]
    
    # Get latest filename
    latest = max(mpath, key=os.path.getmtime)
    print(latest)
    context.logger.debug(f'Reading data from: {latest}')
    
    # Load parquet to dask
    df = dd.read_parquet(latest)
    
    return df

In [30]:
def get_train_test_sets_from_data(context, df: pd.DataFrame):
    drop_columns = [col for col in df.columns if 'is_error' in col]
    X = df.drop(drop_columns, axis=1)
    y = df.loc[:, 'is_error']
    X_train, X_test, y_train, y_test = dcv.train_test_split(X, y, train_size=context.train_size, test_size=1-context.train_size)
    return X_train, X_test, y_train, y_test

### Init context

In [31]:
def init_context(context):
    
    # Netops features table
    setattr(context, 'features_table', os.getenv('FEATURES_TABLE', 'netops_features'))
    
    # Get saving configuration
    is_from_tsdb = (int(os.getenv('FROM_TSDB', 1)) == 1)
    
    # Save to TSDB
    if is_from_tsdb:
        # Create our DB client
        v3io_client = v3f.Client(address='framesd:8081', container='bigdata')
        setattr(context, 'v3f', v3io_client)
        
        # Create features table if neede
        context.v3f.create('tsdb', context.features_table, attrs={'rate': '1/s'}, if_exists=1)
        
        # Set TSDB reading function
        setattr(context, 'read', get_data_tsdb)
        
    # Save to Parquet
    else:
         # Create saving directory if needed
        filepath = os.path.join(context.features_table)
        if not os.path.exists(filepath):
            os.makedirs(filepath)
            
        # Set Parquet reading function
        setattr(context, 'read', get_data_parquet)
    
    # Set time to train on
    train_on_last = os.getenv('TRAIN_ON_LAST', '7d')
    setattr(context, 'train_on_last', train_on_last)
    
    # Set training set size
    train_set_size = float(os.getenv('TRAIN_SIZE', 0.7))
    setattr(context, 'train_size', train_set_size)
    
    # Dask shards / CV
    setattr(context, 'shards', int(os.getenv('NUMBER_OF_SHARDS', 4)))
    
    # Create save-to folder if needed
    model_filepath = os.getenv('SAVE_TO', '/v3io/bigdata/netops/models')
    if not os.path.exists(model_filepath):
        os.makedirs(model_filepath)
    setattr(context, 'model_filepath', os.path.join(model_filepath, os.getenv('MODEL_FILENAME', 'netops.model')))

### Handler

In [32]:
def handler(context, event):
    
    # Get data
    df = context.read(context) 

    # Split to Train / Test datasets
    X_train, X_test, y_train, y_test = get_train_test_sets_from_data(context, df)
    
    # Train
    model = xgb.XGBClassifier()
    model.fit(X_train, y_train)
    
    # Score
    score = model.score(X_test, y_test)
    
    # Save model
    pickle.dump(model, open(context.model_filepath + '.pickle', 'wb'))

## Test

In [41]:
# nuclio: ignore
init_context(context)

In [42]:
# nuclio: ignore
# init_context(context)
event = nuclio.Event(body='')
output = handler(context, event)
output

/v3io/bigdata/netops_features_parquet/20191222T133150-20191222T143000.parquet


## Deployment

In [ ]:
%nuclio deploy -p netops -n training -c